In [ ]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import History

# <b> Import File Path


In [ ]:
dataset = 'dataset/full.csv'
model_save_path = 'keypoint_classifier.keras'

# <b>Set number of classes and random seed

In [ ]:
random_seed = 42
num_classes = 6

# <b> Data Preprocessing

In [ ]:
df = pd.read_csv(dataset)
df

,Label,wristX,wristY,wristZ,thumb_CmcX,thumb_CmcY,thumb_CmcZ,thumb_McpX,thumb_McpY,thumb_McpZ,...,pinky_McpZ,pinky_PipX,pinky_PipY,pinky_PipZ,pinky_DipX,pinky_DipY,pinky_DipZ,pinky_TipX,pinky_TipY,pinky_TipZ
0,0,109.088745,314.879980,-5.032854e-07,151.700001,293.516436,-0.025845,184.029713,249.228258,-0.030217,...,-0.000314,97.632666,209.177756,-0.016920,101.957064,235.674677,-0.003543,99.713421,245.986433,0.011983
1,0,109.720602,314.435120,-5.082854e-07,152.480783,292.276154,-0.025500,183.990440,248.887711,-0.030684,...,-0.001665,97.538586,209.035378,-0.018751,102.451715,235.358377,-0.005369,101.192093,246.964531,0.010406
2,0,104.462366,318.228121,-5.272424e-07,147.859087,298.004780,-0.027812,181.450329,252.615280,-0.031805,...,0.000159,93.628788,212.799540,-0.017536,97.957821,240.048265,-0.004697,96.663094,249.643621,0.010595
3,0,131.711702,412.307510,-3.483320e-07,168.481026,401.964226,-0.024451,194.551659,359.942465,-0.027974,...,-0.007227,116.672125,337.602968,-0.023909,122.332363,359.621401,-0.017577,121.501093,365.053110,-0.008317
4,0,134.165373,410.900230,-3.638212e-07,171.137619,403.897762,-0.026155,198.148975,361.638794,-0.029983,...,-0.007227,121.057186,338.430319,-0.024206,126.223297,360.631142,-0.018549,124.609032,366.415186,-0.009900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,5,211.665516,318.643942,-3.478521e-07,237.260704,312.855806,-0.021787,261.655102,300.156813,-0.032885,...,-0.015840,188.870659,252.664919,-0.025466,182.116089,240.407038,-0.026138,177.263374,228.607378,-0.023531
3596,5,212.169533,318.432026,-3.606120e-07,237.857857,312.789459,-0.022022,262.042198,299.791203,-0.033089,...,-0.016900,188.836765,252.620001,-0.027613,182.190228,240.143623,-0.028949,177.108479,227.796650,-0.026786
3597,5,212.201557,318.900661,-3.305985e-07,237.838631,312.825079,-0.022543,261.752777,299.788485,-0.034097,...,-0.017387,189.020767,252.396469,-0.028404,182.308922,240.092869,-0.029493,177.224693,227.594919,-0.027074
3598,5,212.403927,318.898802,-3.399586e-07,237.947063,312.354841,-0.022142,262.068501,299.327860,-0.033708,...,-0.017171,189.063187,251.963425,-0.027344,182.217598,239.406652,-0.028341,177.025433,227.192116,-0.026079


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 64 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Label        3600 non-null   int64  
 1   wristX       3600 non-null   float64
 2   wristY       3600 non-null   float64
 3   wristZ       3600 non-null   float64
 4   thumb_CmcX   3600 non-null   float64
 5   thumb_CmcY   3600 non-null   float64
 6   thumb_CmcZ   3600 non-null   float64
 7   thumb_McpX   3600 non-null   float64
 8   thumb_McpY   3600 non-null   float64
 9   thumb_McpZ   3600 non-null   float64
 10  thumb_IpX    3600 non-null   float64
 11  thumb_IpY    3600 non-null   float64
 12  thumb_IpZ    3600 non-null   float64
 13  thumb_TipX   3600 non-null   float64
 14  thumb_TipY   3600 non-null   float64
 15  thumb_TipZ   3600 non-null   float64
 16  index_McpX   3600 non-null   float64
 17  index_McpY   3600 non-null   float64
 18  index_McpZ   3600 non-null   float64
 19  index_

# <b> Data loading

In [ ]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype=np.float32, skiprows=1, usecols=range(1, 64)) # Pick the coordinates
y_dataset = np.loadtxt(dataset, delimiter=',', dtype=np.int32, skiprows=1,usecols=(0)) # Pick the label

In [ ]:
print(X_dataset.shape, y_dataset.shape)

(3600, 63) (3600,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=random_seed)

In [ ]:
# Reshape X_train and X_test to have an additional feature dimension (e.g., each timestep has 1 feature)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # (num_samples, timesteps, 1 feature)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))  # (num_samples, timesteps, 1 feature)


# <b> Model Bulding

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Input((63, )),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(20, activation='relu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(6, activation='softmax')
# ])

In [ ]:
# One Dimensional Convolutional Neural Network model, Train will be feed to 1 Dimension Convolutional Neural Network
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=X_train.shape[1:3]),
    tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(rate=0.2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(num_classes, activation='softmax')])

c:\Users\dangq\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()  

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_32 (Conv1D)              │ (None, 63, 32)         │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_33 (Conv1D)              │ (None, 63, 32)         │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_16 (MaxPooling1D) │ (None, 31, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_34 (Conv1D)              │ (None, 31, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_35 (Conv1D)              │ (None, 31, 64)         │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 15, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_36 (Conv1D)              │ (None, 15, 128)        │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_37 (Conv1D)              │ (None, 15, 128)        │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_18 (MaxPooling1D) │ (None, 7, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_38 (Conv1D)              │ (None, 7, 256)         │       164,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_39 (Conv1D)              │ (None, 7, 256)         │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_19 (MaxPooling1D) │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 3, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 512)            │       393,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 6)              │         3,078 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,048,166 (4.00 MB)

 Trainable params: 1,048,166 (4.00 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)

es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# <b> Model Training

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=128,
    validation_data=(X_test, y_test),
    #callbacks=[cp_callback, es_callback],
    callbacks=[History()]
)

Epoch 1/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.2109 - loss: 2.4398 - val_accuracy: 0.3278 - val_loss: 1.6101
Epoch 2/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.4026 - loss: 1.4198 - val_accuracy: 0.5822 - val_loss: 0.7624
Epoch 3/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.6920 - loss: 0.6581 - val_accuracy: 0.9122 - val_loss: 0.2611
Epoch 4/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.9206 - loss: 0.2348 - val_accuracy: 0.9811 - val_loss: 0.0522
Epoch 5/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.9839 - loss: 0.0484 - val_accuracy: 0.9967 - val_loss: 0.0245
Epoch 6/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.9859 - loss: 0.0413 - val_accuracy: 0.9967 - val_loss: 0.0187
Epoch 7/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9917 - loss: 0.0284 - val_accuracy: 0.9956 - val_loss: 0.0154
Epoch 8/100
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9695 - loss: 0.1108 - val_accuracy: 0.

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1: Make predictions on the test set
y_pred_prob = model.predict(X_test)  # Get predicted probabilities for each class

# Step 2: Convert predicted probabilities to predicted class labels
y_pred = np.argmax(y_pred_prob, axis=1)  # Choose the class with the highest probability

# Step 3: Evaluate predictions
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Print classification report (precision, recall, F1-score, etc.)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Accuracy: 0.9989
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       172
           1       0.99      1.00      1.00       153
           2       1.00      0.99      1.00       150
           3       1.00      1.00      1.00       145
           4       1.00      1.00      1.00       144
           5       1.00      1.00      1.00       136

    accuracy                           1.00       900
   macro avg       1.00      1.00      1.00       900
weighted avg       1.00      1.00      1.00       900

Confusion Matrix:
[[172   0   0   0   0   0]
 [  0 153   0   0   0   0]
 [  0   1 149   0   0   0]
 [  0   0   0 145   0   0]
 [  0   0   0   0 144   0]
 [  0   0   0   0   0 136]]
